In [ ]:
# 运行环境 pytorch_env

In [1]:
import cv2
import os
import argparse

In [2]:
output_dir = './images0702'
video_path = './my_video-3.mkv'

os.makedirs(output_dir, exist_ok=True)
left_dir = os.path.join(output_dir, 'left')
right_dir = os.path.join(output_dir, 'right')
os.makedirs(left_dir, exist_ok=True)
os.makedirs(right_dir, exist_ok=True)

# 打开视频文件
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("无法打开视频文件")

In [3]:
# 获取视频信息
fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

print(f"视频信息: {total_frames}帧 | {fps:.2f}FPS")
print(f"开始处理，输出目录: {output_dir}")

frame_count = 0
success = True

while success:
    success, frame = cap.read()
    if not success:
        break
        
    # 获取帧尺寸并计算中心分割线
    height, width = frame.shape[:2]
    mid_point = width // 2
    
    # 分割左右帧
    left_frame = frame[:, :mid_point]
    right_frame = frame[:, mid_point:]
    
    # 保存图像
    frame_filename = f"frame_{frame_count:06d}.jpg"
    cv2.imwrite(os.path.join(left_dir, frame_filename), left_frame)
    cv2.imwrite(os.path.join(right_dir, frame_filename), right_frame)
    
    frame_count += 1
    
    # 每100帧打印进度
    if frame_count % 100 == 0:
        print(f"已处理 {frame_count}/{total_frames} 帧...")

cap.release()
print(f"处理完成! 共保存 {frame_count} 帧")
print(f"左视图保存至: {left_dir}")
print(f"右视图保存至: {right_dir}")

视频信息: 3611帧 | 30.00FPS
开始处理，输出目录: ./images0702
已处理 100/3611 帧...
已处理 200/3611 帧...
已处理 300/3611 帧...
已处理 400/3611 帧...
已处理 500/3611 帧...
已处理 600/3611 帧...
已处理 700/3611 帧...
已处理 800/3611 帧...
已处理 900/3611 帧...
已处理 1000/3611 帧...
已处理 1100/3611 帧...
已处理 1200/3611 帧...
已处理 1300/3611 帧...
已处理 1400/3611 帧...
已处理 1500/3611 帧...
已处理 1600/3611 帧...
已处理 1700/3611 帧...
已处理 1800/3611 帧...
已处理 1900/3611 帧...
已处理 2000/3611 帧...
已处理 2100/3611 帧...
已处理 2200/3611 帧...
处理完成! 共保存 2290 帧
左视图保存至: ./images0702\left
右视图保存至: ./images0702\right


## 移动同名文件

In [30]:
from collections import Counter
import shutil

In [12]:
files = os.listdir('./images/left')
files_drop_type = [x.split('.')[0] for x in files]

counter = Counter(files_drop_type)
counter

Counter({'frame_001107': 2,
         'frame_001108': 2,
         'frame_001109': 2,
         'frame_001110': 1,
         'frame_001111': 1,
         'frame_001112': 1,
         'frame_001113': 2,
         'frame_001114': 1,
         'frame_001115': 1,
         'frame_001116': 1,
         'frame_001117': 1,
         'frame_001118': 2,
         'frame_001119': 1,
         'frame_001120': 1,
         'frame_001121': 1,
         'frame_001122': 2,
         'frame_001123': 1,
         'frame_001124': 1,
         'frame_001125': 1,
         'frame_001126': 2,
         'frame_001127': 1,
         'frame_001128': 1,
         'frame_001129': 1,
         'frame_001130': 2,
         'frame_001131': 1,
         'frame_001132': 1,
         'frame_001133': 1,
         'frame_001134': 2,
         'frame_001135': 1,
         'frame_001136': 1,
         'frame_001137': 1,
         'frame_001138': 2,
         'frame_001139': 1,
         'frame_001140': 1,
         'frame_001141': 1,
         'frame_0011

In [33]:
for item in list(dict(counter).keys()):
    num = counter[item]

    if num == 2:
        # 源文件路径
        src_jpg_file = f'./images/left/{item}.jpg'
        src_json_file = f'./images/left/{item}.json'

        # 目标路径（可以是文件夹，也可以包含目标文件名）
        dst_jpg_path = f'./images/left_keep/{item}.jpg'  
        dst_json_path = f'./images/left_keep/{item}.json'  

        # 拷贝文件
        shutil.copy(src_jpg_file, dst_jpg_path)
        shutil.copy(src_json_file, dst_json_path)

    continue


In [13]:
counter['frame_001107']

2

## 移动文件

In [36]:
files = os.listdir('./images/left_keep')
for item in files:
    tp = item.split('.')[-1]

    if tp == 'jpg':
        src_jpg_file = f'./images/left_keep/{item}'
        dst_jpg_path = f'./XHY/images/{item}'  
        shutil.copy(src_jpg_file, dst_jpg_path)
    elif tp == 'json':
        src_json_file = f'./images/left_keep/{item}'
        dst_json_path = f'./XHY/jsons/{item}'  
        shutil.copy(src_json_file, dst_json_path)
    else:
        print(item)



## 数据类型转换

In [ ]:
import json
import os
from PIL import Image

# 路径配置
json_dir = './XHY/jsons'
img_dir = './XHY/images'
label_output_dir = './XHY/labels'

os.makedirs(label_output_dir, exist_ok=True)

# 类别名 → ID 映射（确保与你训练用的类别一致）
class_name_to_id = {
    "red_ball": 0,
    "blue_ball": 1,
    # 添加你自己用的类别名
}

def convert_labelme_json_to_yolo(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)

    image_path = os.path.join(img_dir, os.path.basename(data['imagePath']))
    image = Image.open(image_path)
    W, H = image.size

    yolo_lines = []
    for shape in data['shapes']:
        label = shape['label']
        if label not in class_name_to_id:
            continue
        class_id = class_name_to_id[label]

        points = shape['points']
        x_coords = [p[0] for p in points]
        y_coords = [p[1] for p in points]

        xmin = min(x_coords)
        xmax = max(x_coords)
        ymin = min(y_coords)
        ymax = max(y_coords)

        x_center = (xmin + xmax) / 2 / W
        y_center = (ymin + ymax) / 2 / H
        width = (xmax - xmin) / W
        height = (ymax - ymin) / H

        yolo_lines.append(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

    # 输出到同名.txt文件
    label_name = os.path.splitext(os.path.basename(json_path))[0] + '.txt'
    label_path = os.path.join(label_output_dir, label_name)

    with open(label_path, 'w') as f:
        f.write('\n'.join(yolo_lines))

# 批量转换
for filename in os.listdir(json_dir):
    if filename.endswith('.json'):
        convert_labelme_json_to_yolo(os.path.join(json_dir, filename))

print("✅ 转换完成，YOLO格式标签保存在 labels/ 文件夹中")
